In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project="ee-gaoliaoran2020")

In [ ]:
# import geemap
import geemap
import datetime
import numpy as np
import math
from math import nan
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
import matplotlib.font_manager as fm
font_paths = fm.findSystemFonts()
font_paths

In [ ]:
import matplotlib as mpl
mpl.rcParams["font.serif"]

In [ ]:
Supervised_classification = ee.Image("projects/ee-gaoliaoran2020/assets/classification_all_series")
probability_all_series = ee.Image("projects/ee-gaoliaoran2020/assets/probility_all_series")
four = ee.FeatureCollection("projects/ee-gaoliaoran2020/assets/zhangZT/2020_the_second_method/four")
LULC = ee.ImageCollection("ESA/WorldCover/v100").first().clip(four);
image_cropland = LULC.updateMask(LULC.eq(40)).multiply(0).add(1)
probability_max = probability_all_series.reduce(ee.Reducer.max())
classification_cropland = Supervised_classification.add(1).multiply(image_cropland)

In [ ]:
def mask_type(image,my_type):
  mask=image.eq(my_type);
  masked_image=image.updateMask(mask);
  return masked_image;

In [ ]:
corn_area = mask_type(classification_cropland, 1)
corn_wheat_area = mask_type(classification_cropland, 2)
cotton_area = mask_type(classification_cropland, 3)
wheat_area = mask_type(classification_cropland, 4)
fruit_area = mask_type(classification_cropland, 5)

In [ ]:
Map = geemap.Map()
Map

In [ ]:
# Map.addLayer(cotton_area.multiply(0).add(1))
# mask_probability_start=probability_max.multiply(cotton_area).gte(0.6).selfMask()
# mask_probability_end=probability_max.multiply(cotton_area).lt(0.7).selfMask()
# probability = mask_probability_start.add(mask_probability_end)
# Map.addLayer(mask_probability_end)

In [ ]:
def pixel_count_probability_class(class_image, probability_start, probablility_end):
  class_image = class_image.multiply(0).add(1)
  mask_probability_start = probability_max.multiply(class_image).gte(probability_start).selfMask()
  mask_probability_end = probability_max.multiply(class_image).lt(probablility_end).selfMask()
  probability = mask_probability_start.add(mask_probability_end).eq(2).selfMask()
  countDictionary = probability.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=four.geometry(),
    scale=10,
    maxPixels=1e13
    );
  number = countDictionary.getInfo()

  value = number["max"]
  print('cropland',value);
  return value

In [ ]:
area = corn_area
image_00_01 = pixel_count_probability_class(area, 0.0, 0.1)
image_01_02 = pixel_count_probability_class(area, 0.1, 0.2)
image_02_03 = pixel_count_probability_class(area, 0.2, 0.3)
image_03_04 = pixel_count_probability_class(area, 0.3, 0.4)
image_04_05 = pixel_count_probability_class(area, 0.4, 0.5)
image_05_06 = pixel_count_probability_class(area, 0.5, 0.6)
image_06_07 = pixel_count_probability_class(area, 0.6, 0.7)
image_07_08 = pixel_count_probability_class(area, 0.7, 0.8)
image_08_09 = pixel_count_probability_class(area, 0.8, 0.9)
image_09_10 = pixel_count_probability_class(area, 0.9, 1.0)
image = [image_00_01,image_01_02,image_02_03,image_03_04,image_04_05,image_05_06,image_06_07,image_07_08,image_08_09,image_09_10]

In [ ]:
image = [0,0,2100761,13152847,4286894,3396940,1767565,784,0,0]

In [ ]:
import csv
import os, sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
import math

In [ ]:
# 打开矢量
csv_origin_dir = r"/content/drive/MyDrive/probility"
os.chdir(csv_origin_dir)
CSV_list = glob.glob("*.csv")
print(CSV_list)
# -----------------------------------------------------------
i = 5
# ------------------------------------------------------------
print(CSV_list[i])
filename =os.path.join(csv_origin_dir, CSV_list[i])
data = pd.read_csv(filename)

cornData = np.array(data['max'].values)
cornData

In [ ]:
def count_sample(class_data, probability_start, probablility_end):
  number = 0
  for i in range(len(class_data)):
    data = class_data[i]
    if data >= probability_start and data < probablility_end:
      number = number+1
    else:
      continue
  return number

In [ ]:
Data = cornData
point_00_01 = count_sample(Data, 0.0, 0.1)
point_01_02 = count_sample(Data, 0.1, 0.2)
point_02_03 = count_sample(Data, 0.2, 0.3)
point_03_04 = count_sample(Data, 0.3, 0.4)
point_04_05 = count_sample(Data, 0.4, 0.5)
point_05_06 = count_sample(Data, 0.5, 0.6)
point_06_07 = count_sample(Data, 0.6, 0.7)
point_07_08 = count_sample(Data, 0.7, 0.8)
point_08_09 = count_sample(Data, 0.8, 0.9)
point_09_10 = count_sample(Data, 0.9, 1.0)
point = [point_00_01,point_01_02,point_02_03,point_03_04,point_04_05,point_05_06,point_06_07,point_07_08,point_08_09,point_09_10]

In [ ]:
image_percent = [x/sum(image) for x in image]
point_percent = [x/sum(point) for x in point]

In [ ]:
image_percent

In [ ]:
point_percent

In [ ]:
D = 0
end = 6
new_image = image[:-3]
new_point_percent = point_percent[:-3]
for j in range(0,end):
  new_image[j] = 0
  new_image_percent = [x/sum(new_image) for x in new_image]
  D = 0
  # print(new_image_percent)
  # print(new_point_percent)
  for i in range(0,len(new_image_percent)):
    x = (abs(point_percent[i]-new_image_percent[i])-0.1)**2
    D = D+x
  print(D)

In [ ]:
corn_list = [0.20864625064637196,0.20864625064637196,0.26922854061134216,0.062472070011341196,0.38834467120181415]
corn_difference = [0]
for i in range(1,len(corn_list)):
  if corn_list[i]-corn_list[i-1] == 0:
    corn_difference.append(0)
  elif corn_list[i]-corn_list[i-1] > 0:
    corn_difference.append(1)
  else:
    corn_difference.append(-1)
i = [0.1,0.2,0.3,0.4,0.5]
corn_difference

In [ ]:
# Create some mock data
fig, ax1 = plt.subplots()
mpl.rc('font', family='serif')
plt.style.use('bmh')
color = 'tab:blue'
ax1.set_xlabel('$k$',size=20)
ax1.set_ylabel('$D_k$',color=color,size=20)
lns1 =ax1.plot(i,corn_list,color=color,label='$D_k$')
ax1.tick_params(axis='y',labelcolor=color,labelsize=16)
ax1.tick_params(axis='x',labelcolor=color,labelsize=16)
ax1.set_title("Corn",size=20)

ax2 = ax1.twinx()

color = 'tab:red'
ax2.set_ylabel('$|D_k|/D_k$',color=color,size=20)
lns2 =ax2.plot(i, corn_difference,color=color,label='$|D_k|/D_k$')
ax2.tick_params(axis='y',labelcolor=color,labelsize=16)

lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns,labs,loc=0,prop={'size':14})

fig.tight_layout()
plt.savefig('corn.png',dpi=330,format='png',bbox_inches='tight')
files.download('corn.png')

In [ ]:
image_00_01 = pixel_count_probability_class(cotton_area, 0.0, 0.1)
image_01_02 = pixel_count_probability_class(cotton_area, 0.1, 0.2)
image_02_03 = pixel_count_probability_class(cotton_area, 0.2, 0.3)
image_03_04 = pixel_count_probability_class(cotton_area, 0.3, 0.4)
image_04_05 = pixel_count_probability_class(cotton_area, 0.4, 0.5)
image_05_06 = pixel_count_probability_class(cotton_area, 0.5, 0.6)
image_06_07 = pixel_count_probability_class(cotton_area, 0.6, 0.7)
image_07_08 = pixel_count_probability_class(cotton_area, 0.7, 0.8)
image_08_09 = pixel_count_probability_class(cotton_area, 0.8, 0.9)
image_09_10 = pixel_count_probability_class(cotton_area, 0.9, 1.0)
image = [image_00_01,image_01_02,image_02_03,image_03_04,image_04_05,image_05_06,image_06_07,image_07_08,image_08_09,image_09_10]

In [ ]:
image = [0,0,1194355,6916834,9029750,15611966,13354467,398683,0,0]

In [ ]:
# 打开矢量
csv_origin_dir = r"/content/drive/MyDrive/probility"
os.chdir(csv_origin_dir)
CSV_list = glob.glob("*.csv")
print(CSV_list)
# -----------------------------------------------------------
i = 6
# ------------------------------------------------------------
print(CSV_list[i])
filename =os.path.join(csv_origin_dir, CSV_list[i])
data = pd.read_csv(filename)

cottonData = np.array(data['max'].values)
cottonData

In [ ]:
Data = cottonData
point_00_01 = count_sample(Data, 0.0, 0.1)
point_01_02 = count_sample(Data, 0.1, 0.2)
point_02_03 = count_sample(Data, 0.2, 0.3)
point_03_04 = count_sample(Data, 0.3, 0.4)
point_04_05 = count_sample(Data, 0.4, 0.5)
point_05_06 = count_sample(Data, 0.5, 0.6)
point_06_07 = count_sample(Data, 0.6, 0.7)
point_07_08 = count_sample(Data, 0.7, 0.8)
point_08_09 = count_sample(Data, 0.8, 0.9)
point_09_10 = count_sample(Data, 0.9, 1.0)
point = [point_00_01,point_01_02,point_02_03,point_03_04,point_04_05,point_05_06,point_06_07,point_07_08,point_08_09,point_09_10]

In [ ]:
image_percent = [x/sum(image) for x in image]
image_percent

In [ ]:
point_percent = [x/sum(point) for x in point]
point_percent

In [ ]:
D = 0
end = 6
new_image = image[:-3]
new_point_percent = point_percent[:-3]
for j in range(0,end):
  new_image[j] = 0
  new_image_percent = [x/sum(new_image) for x in new_image]
  D = 0
  # print(new_image_percent)
  # print(new_point_percent)
  for i in range(0,len(new_image_percent)):
    x = (abs(point_percent[i]-new_image_percent[i])-0.1)**2
    D = D+x
  print(D)

In [ ]:
cotton_list = [0.05277765122889101,0.05277765122889101,0.05010431957519463,0.07260172022371561,0.21963718820861683]
cotton_difference = [0]
for i in range(1,len(cotton_list)):
  if cotton_list[i]-cotton_list[i-1] == 0:
    cotton_difference.append(0)
  elif cotton_list[i]-cotton_list[i-1] > 0:
    cotton_difference.append(1)
  else:
    cotton_difference.append(-1)
i = [0.1,0.2,0.3,0.4,0.5]
cotton_difference

In [ ]:
# Create some mock data
fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('$k$',size=20)
ax1.set_ylabel('$D_k$',color=color,size=20)
lns1 =ax1.plot(i,cotton_list,color=color,label='$D_k$')
ax1.tick_params(axis='y',labelcolor=color,labelsize=16)
ax1.tick_params(axis='x',labelcolor=color,labelsize=16)
ax1.set_title("Cotton",size=20)

ax2 = ax1.twinx()

color = 'tab:red'
ax2.set_ylabel('$|D_k|/D_k$',color=color,size=20)
lns2 =ax2.plot(i, cotton_difference,color=color,label='$|D_k|/D_k$')
ax2.tick_params(axis='y',labelcolor=color,labelsize=16)

lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns,labs,loc=0,prop={'size':14})

fig.tight_layout()
plt.savefig('cotton.png',dpi=330,format='png',bbox_inches='tight')
files.download('cotton.png')

In [ ]:
area = fruit_area
image_00_01 = pixel_count_probability_class(area, 0.0, 0.1)
image_01_02 = pixel_count_probability_class(area, 0.1, 0.2)
image_02_03 = pixel_count_probability_class(area, 0.2, 0.3)
image_03_04 = pixel_count_probability_class(area, 0.3, 0.4)
image_04_05 = pixel_count_probability_class(area, 0.4, 0.5)
image_05_06 = pixel_count_probability_class(area, 0.5, 0.6)
image_06_07 = pixel_count_probability_class(area, 0.6, 0.7)
image_07_08 = pixel_count_probability_class(area, 0.7, 0.8)
image_08_09 = pixel_count_probability_class(area, 0.8, 0.9)
image_09_10 = pixel_count_probability_class(area, 0.9, 1.0)
image = [image_00_01,image_01_02,image_02_03,image_03_04,image_04_05,image_05_06,image_06_07,image_07_08,image_08_09,image_09_10]

In [ ]:
# 打开矢量
csv_origin_dir = r"/content/drive/MyDrive/probility"
os.chdir(csv_origin_dir)
CSV_list = glob.glob("*.csv")
print(CSV_list)
# -----------------------------------------------------------
i = 7
# ------------------------------------------------------------
print(CSV_list[i])
filename =os.path.join(csv_origin_dir, CSV_list[i])
data = pd.read_csv(filename)

forestData = np.array(data['max'].values)
forestData

In [ ]:
Data = forestData
point_00_01 = count_sample(Data, 0.0, 0.1)
point_01_02 = count_sample(Data, 0.1, 0.2)
point_02_03 = count_sample(Data, 0.2, 0.3)
point_03_04 = count_sample(Data, 0.3, 0.4)
point_04_05 = count_sample(Data, 0.4, 0.5)
point_05_06 = count_sample(Data, 0.5, 0.6)
point_06_07 = count_sample(Data, 0.6, 0.7)
point_07_08 = count_sample(Data, 0.7, 0.8)
point_08_09 = count_sample(Data, 0.8, 0.9)
point_09_10 = count_sample(Data, 0.9, 1.0)
point = [point_00_01,point_01_02,point_02_03,point_03_04,point_04_05,point_05_06,point_06_07,point_07_08,point_08_09,point_09_10]

In [ ]:
image_percent = [x/sum(image) for x in image]
image_percent

In [ ]:
point_percent = [x/sum(point) for x in point]
point_percent

In [ ]:
D = 0
end = 5
new_image = image[:-4]
new_point_percent = point_percent[:-4]
for j in range(0,end):
  new_image[j] = 0
  new_image_percent = [x/sum(new_image) for x in new_image]
  D = 0
  # print(new_image_percent)
  # print(new_point_percent)
  for i in range(0,len(new_image_percent)):
    x = (abs(point_percent[i]-new_image_percent[i])-0.1)**2
    D = D+x
  print(D)

In [ ]:
forest_list = [0.05205457113096666,0.05205457113096666,0.09487496293787434,0.2852315792348148,0.7495061728395062]
forest_difference = [0]
for i in range(1,len(forest_list)):
  if forest_list[i]-forest_list[i-1] == 0:
    forest_difference.append(0)
  elif forest_list[i]-forest_list[i-1] > 0:
    forest_difference.append(1)
  else:
    forest_difference.append(-1)
i = [0.1,0.2,0.3,0.4,0.5]
forest_difference

In [ ]:
# Create some mock data
fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('$k$',size=20)
ax1.set_ylabel('$D_k$',color=color,size=20)
lns1 =ax1.plot(i,forest_list,color=color,label='$D_k$')
ax1.tick_params(axis='y',labelcolor=color,labelsize=16)
ax1.tick_params(axis='x',labelcolor=color,labelsize=16)
ax1.set_title("Orchard",size=20)

ax2 = ax1.twinx()

color = 'tab:red'
ax2.set_ylabel('$|D_k|/D_k$',color=color,size=20)
lns2 =ax2.plot(i, forest_difference,color=color,label='$|D_k|/D_k$')
ax2.tick_params(axis='y',labelcolor=color,labelsize=16)

lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns,labs,loc=0,prop={'size':14})

fig.tight_layout()
plt.savefig('Orchard.png',dpi=330,format='png',bbox_inches='tight')
files.download('Orchard.png')

In [ ]:
area = wheat_area
image_00_01 = pixel_count_probability_class(area, 0.0, 0.1)
image_01_02 = pixel_count_probability_class(area, 0.1, 0.2)
image_02_03 = pixel_count_probability_class(area, 0.2, 0.3)
image_03_04 = pixel_count_probability_class(area, 0.3, 0.4)
image_04_05 = pixel_count_probability_class(area, 0.4, 0.5)
image_05_06 = pixel_count_probability_class(area, 0.5, 0.6)
image_06_07 = pixel_count_probability_class(area, 0.6, 0.7)
image_07_08 = pixel_count_probability_class(area, 0.7, 0.8)
image_08_09 = pixel_count_probability_class(area, 0.8, 0.9)
image_09_10 = pixel_count_probability_class(area, 0.9, 1.0)
image = [image_00_01,image_01_02,image_02_03,image_03_04,image_04_05,image_05_06,image_06_07,image_07_08,image_08_09,image_09_10]

In [ ]:
# 打开矢量
csv_origin_dir = r"/content/drive/MyDrive/probility"
os.chdir(csv_origin_dir)
CSV_list = glob.glob("*.csv")
print(CSV_list)
# -----------------------------------------------------------
i = 9
# ------------------------------------------------------------
print(CSV_list[i])
filename =os.path.join(csv_origin_dir, CSV_list[i])
data = pd.read_csv(filename)

wheatData = np.array(data['max'].values)
wheatData

In [ ]:
Data = wheatData
point_00_01 = count_sample(Data, 0.0, 0.1)
point_01_02 = count_sample(Data, 0.1, 0.2)
point_02_03 = count_sample(Data, 0.2, 0.3)
point_03_04 = count_sample(Data, 0.3, 0.4)
point_04_05 = count_sample(Data, 0.4, 0.5)
point_05_06 = count_sample(Data, 0.5, 0.6)
point_06_07 = count_sample(Data, 0.6, 0.7)
point_07_08 = count_sample(Data, 0.7, 0.8)
point_08_09 = count_sample(Data, 0.8, 0.9)
point_09_10 = count_sample(Data, 0.9, 1.0)
point = [point_00_01,point_01_02,point_02_03,point_03_04,point_04_05,point_05_06,point_06_07,point_07_08,point_08_09,point_09_10]

In [ ]:
image_percent = [x/sum(image) for x in image]
image_percent

In [ ]:
point_percent = [x/sum(point) for x in point]
point_percent

In [ ]:
D = 0
end = 5
new_image = image[:-4]
new_point_percent = point_percent[:-4]
for j in range(0,end):
  new_image[j] = 0
  new_image_percent = [x/sum(new_image) for x in new_image]
  D = 0
  # print(new_image_percent)
  # print(new_point_percent)
  for i in range(0,len(new_image_percent)):
    x = (abs(point_percent[i]-new_image_percent[i])-0.1)**2
    D = D+x
  print(D)

In [ ]:
wheat_list = [0.07002649347882511,0.07002649347882511,0.035186804124346764,0.2512111553667974,1.1600000000000001]
wheat_difference = [0]
for i in range(1,len(wheat_list)):
  if wheat_list[i]-wheat_list[i-1] == 0:
    wheat_difference.append(0)
  elif wheat_list[i]-wheat_list[i-1] > 0:
    wheat_difference.append(1)
  else:
    wheat_difference.append(-1)
i = [0.1,0.2,0.3,0.4,0.5]
wheat_difference

In [ ]:
# Create some mock data
fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('$k$',size=20)
ax1.set_ylabel('$D_k$',color=color,size=20)
lns1 =ax1.plot(i,wheat_list,color=color,label='$D_k$')
ax1.tick_params(axis='y',labelcolor=color,labelsize=16)
ax1.tick_params(axis='x',labelcolor=color,labelsize=16)
ax1.set_title("Wheat",size=20)

ax2 = ax1.twinx()

color = 'tab:red'
ax2.set_ylabel('$|D_k|/D_k$',color=color,size=20)
lns2 =ax2.plot(i, wheat_difference,color=color,label='$|D_k|/D_k$')
ax2.tick_params(axis='y',labelcolor=color,labelsize=16)

lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns,labs,loc=0,prop={'size':14})

fig.tight_layout()
plt.savefig('wheat.png',dpi=330,format='png',bbox_inches='tight')
files.download('wheat.png')

In [ ]:
area = corn_wheat_area
image_00_01 = pixel_count_probability_class(area, 0.0, 0.1)
image_01_02 = pixel_count_probability_class(area, 0.1, 0.2)
image_02_03 = pixel_count_probability_class(area, 0.2, 0.3)
image_03_04 = pixel_count_probability_class(area, 0.3, 0.4)
image_04_05 = pixel_count_probability_class(area, 0.4, 0.5)
image_05_06 = pixel_count_probability_class(area, 0.5, 0.6)
image_06_07 = pixel_count_probability_class(area, 0.6, 0.7)
image_07_08 = pixel_count_probability_class(area, 0.7, 0.8)
image_08_09 = pixel_count_probability_class(area, 0.8, 0.9)
image_09_10 = pixel_count_probability_class(area, 0.9, 1.0)
image = [image_00_01,image_01_02,image_02_03,image_03_04,image_04_05,image_05_06,image_06_07,image_07_08,image_08_09,image_09_10]

In [ ]:
# 打开矢量
csv_origin_dir = r"/content/drive/MyDrive/probility"
os.chdir(csv_origin_dir)
CSV_list = glob.glob("*.csv")
print(CSV_list)
# -----------------------------------------------------------
i = 8
# ------------------------------------------------------------
print(CSV_list[i])
filename =os.path.join(csv_origin_dir, CSV_list[i])
data = pd.read_csv(filename)

corn_wheatData = np.array(data['max'].values)
corn_wheatData

In [ ]:
Data = corn_wheatData
point_00_01 = count_sample(Data, 0.0, 0.1)
point_01_02 = count_sample(Data, 0.1, 0.2)
point_02_03 = count_sample(Data, 0.2, 0.3)
point_03_04 = count_sample(Data, 0.3, 0.4)
point_04_05 = count_sample(Data, 0.4, 0.5)
point_05_06 = count_sample(Data, 0.5, 0.6)
point_06_07 = count_sample(Data, 0.6, 0.7)
point_07_08 = count_sample(Data, 0.7, 0.8)
point_08_09 = count_sample(Data, 0.8, 0.9)
point_09_10 = count_sample(Data, 0.9, 1.0)
point = [point_00_01,point_01_02,point_02_03,point_03_04,point_04_05,point_05_06,point_06_07,point_07_08,point_08_09,point_09_10]

In [ ]:
image_percent = [x/sum(image) for x in image]
image_percent

In [ ]:
point_percent = [x/sum(point) for x in point]
point_percent

In [ ]:
D = 0
end = 5
new_image = image[:-4]
new_point_percent = point_percent[:-4]
for j in range(0,end):
  new_image[j] = 0
  new_image_percent = [x/sum(new_image) for x in new_image]
  D = 0
  # print(new_image_percent)
  # print(new_point_percent)
  for i in range(0,len(new_image_percent)):
    x = (abs(point_percent[i]-new_image_percent[i])-0.1)**2
    D = D+x
  print(D)

In [ ]:
corn_wheat_list = [0.036106970707934755,0.036106970707934755,0.02511374505609646,0.4298969404209922,0.88109375]
corn_wheat_difference = [0]
for i in range(1,len(corn_wheat_list)):
  if corn_wheat_list[i]-corn_wheat_list[i-1] == 0:
    corn_wheat_difference.append(0)
  elif corn_wheat_list[i]-corn_wheat_list[i-1] > 0:
    corn_wheat_difference.append(1)
  else:
    corn_wheat_difference.append(-1)
i = [0.1,0.2,0.3,0.4,0.5]
corn_wheat_difference

In [ ]:
# Create some mock data
fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('$k$',size=20)
ax1.set_ylabel('$D_k$',color=color,size=20)
lns1 =ax1.plot(i,corn_wheat_list,color=color,label='$D_k$')
ax1.tick_params(axis='y',labelcolor=color,labelsize=16)
ax1.tick_params(axis='x',labelcolor=color,labelsize=16)
ax1.set_title("Wheat-Corn",size=20)

ax2 = ax1.twinx()

color = 'tab:red'
ax2.set_ylabel('$|D_k|/D_k$',color=color,size=20)
lns2 =ax2.plot(i, corn_wheat_difference,color=color,label='$|D_k|/D_k$')
ax2.tick_params(axis='y',labelcolor=color,labelsize=16)

lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns,labs,loc=0,prop={'size':14})

fig.tight_layout()
plt.savefig('corn_wheat.png',dpi=330,format='png',bbox_inches='tight')
files.download('corn_wheat.png')